# <div align='center'> Pi 4B </div>

## 系统配置(Pi 4B/8G)

烧录系统版本: https://downloads.raspberrypi.org/raspios_lite_arm64/images/raspios_lite_arm64-2020-08-24/

pi: k12ai (默认: pi)

root: k12ai (默认: raspberrypi)

### 网路配置

-  `vi /etc/dhcpcd.conf`

    ```
    interface eth0
    static ip_address=172.16.1.34/24
    #static ip6_address=fd51:42f8:caae:d92e::ff/64
    static routers=172.16.1.1
    static domain_name_servers=219.141.140.10 219.141.136.10
    
    interface wlan0
    static ip_address=172.16.1.36/24
    #static ip6_address=fd51:42f8:caae:d92e::ff/64
    static routers=172.16.1.1
    static domain_name_servers=219.141.140.10 219.141.136.10
    ```
    
- `vi /etc/wpa_supplicant/wpa_supplicant.conf`

    ```
    ctrl_interface=DIR=/var/run/wpa_supplicant GROUP=netdev
    update_config=1
    country=CN

    network={
        ssid="国电社区"
        psk="88888888"
        key_mgmt=WPA-PSK
    }
    ```

### 时区/语言

可以通过`sudo raspi-config`设置

### 图形桌面(lite to kde)

- `sudo apt-get install lightdm`

- `sudo apt install mate-desktop-environment-core`

- `sudo raspi-config`设置启动模式

### VNC View

- enable: `sudo raspi-config`

- client: `https://www.realvnc.com/en/connect/download/viewer/linux`

### SSH配置

可以通过`sudo raspi-config`设置

免密登陆: `ssh-copy-id -i /home/lidong/.ssh/id_rsa_raspberry.pub pi@172.16.1.34`

### 修改数据源

[更多](https://blog.csdn.net/zqxdsy/article/details/102574239)

- apt: `vi /etc/apt/sources.list.d/raspi.list`

    ```
    deb https://mirrors.tuna.tsinghua.edu.cn/debian/ buster main contrib non-free
    # deb-src https://mirrors.tuna.tsinghua.edu.cn/debian/ buster main contrib non-free
    deb https://mirrors.tuna.tsinghua.edu.cn/debian/ buster-updates main contrib non-free
    # deb-src https://mirrors.tuna.tsinghua.edu.cn/debian/ buster-updates main contrib non-free
    deb https://mirrors.tuna.tsinghua.edu.cn/debian/ buster-backports main contrib non-free
    # deb-src https://mirrors.tuna.tsinghua.edu.cn/debian/ buster-backports main contrib non-free
    deb https://mirrors.tuna.tsinghua.edu.cn/debian-security buster/updates main contrib non-free
    # deb-src https://mirrors.tuna.tsinghua.edu.cn/debian-security buster/updates main contrib non-free
    ```

- pip: `vi ~/.pip/pip.conf`
    
    ```
    [global]
        timeout = 6000
        index-url = http://mirrors.aliyun.com/pypi/simple
        # index-url = http://pypi.douban.com/simple/
    [install]
        use-mirrors = true
        mirrors = http://mirrors.aliyun.com/pypi/simple
        trusted-host = mirrors.aliyun.com
        # mirrors = http://pypi.douban.com/simple/
        # trusted-host = pypi.douban.com
    ```

### Other

```
pip3 install gsutil --user
```

```
sudo apt install libopenblas-dev libblas-dev m4 
sudo apt install libtiff5-dev libjpeg62-turbo-dev libopenjp2-7-dev zlib1g-dev
sudo apt install libfreetype6-dev liblcms2-dev libwebp-dev tcl8.6-dev tk8.6-dev
sudo apt install python3-tk libharfbuzz-dev libfribidi-dev libxcb1-dev
sudo apt install libavutil-dev libavcodec-dev libavformat-dev libswscale-dev
```

```
mkdir pytorch_install
cd pytorch_install
git clone --recursive https://github.com/pytorch/pytorch
cd pytorch
git submodule update --remote third_party/protobuf
export NO_CUDA=1
export NO_DISTRIBUTED=1
export NO_MKLDNN=1
export NO_NNPACK=1
export NO_QNNPACK=1
python setup.py build
sudo -E python setup.py install
cd ..
mkdir pytorch_vision_install && cd pytorch_vision_install
git clone --recursive https://github.com/pytorch/vision
cd vision
sudo -E python setup.py install
```

### pytorch

https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html

## 交叉编译环境

[传送门1](https://medium.com/@stonepreston/how-to-cross-compile-a-cmake-c-application-for-the-raspberry-pi-4-on-ubuntu-20-04-bac6735d36df)

[传送门2](https://tttapa.github.io/Pages/Raspberry-Pi/C++-Development/Building-The-Toolchain.html)

### 目标机(Target Pi4)环境
  
- 安装

  - 二进制工具集:｀sudo apt install -y binutils-source`
  
  > ls /usr/src/binutils/patches/
  
  - 绝对路径转相对: `sudo apt install -y symlinks`
  
  > sudo symlinks -rc /

- 查看系统信息
  
  - kernel版本: `uname -a`
  
  > Linux raspberrypi **5.4.51-v8+** #1333 SMP PREEMPT Mon Aug 10 16:58:35 BST 2020 aarch64 GNU/Linux
  
  - binutil(linker)版本: `ld --version`
  
  > GNU ld (GNU Binutils for Debian) 2.31.1
  
  - gcc版本: `gcc --version`
  
  > gcc (Debian 8.3.0-6) 8.3.0
  
  - GLIBC版本: `ldd --version`
  
  > ldd (Debian GLIBC 2.28-10) 2.28
  
**所有信息在配置menuconfig上使用**, cortex a53(2核) a72(4核)

### 主机(Host)环境

1. crosstool-ng依赖库

  ```
  sudo apt install -y gcc g++ gperf bison flex texinfo help2man cmake-curses-gui make \
    libncurses5-dev python3-dev autoconf automake libtool libtool-bin \
    gawk wget bzip2 xz-utils unzip patch libstdc++6 rsync git
  ```
  
- 安装crosstool-ng

  [tarball](http://crosstool-ng.org/download/crosstool-ng)
 
  - `cd /data/raspberry_pi4`
  - `wget http://crosstool-ng.org/download/crosstool-ng/crosstool-ng-1.24.0.tar.bz2`
  - `tar xjf crosstool-ng-1.24.0.tar.bz2`
  - `cd crosstool-ng-1.24.0`
  - `./configure --prefix=/opt/x-tools`
  - `make -j4; sudo make install`
  
  or
  
  - `git clone --depth 1 https://github.com/crosstool-ng/crosstool-ng.git`
  - `cd crosstool-ng; ./bootstrap`
  - `./configure --prefix=/opt/x-tools`
  - `make -j4; sudo make install`

- 设置crosstoll-ng: `echo "export PATH=/opt/x-tools/bin:$PATH" >> ~/.bashrc`

### 编译交叉编译链

- 创建staging目录
  
  - `mkdir -p /data/raspberry_pi4/toolchain_staging`
  - `cd /data/raspberry_pi4/toolchain_staging`
  - `mkdir -p src patches/binutils/2.31.1/`
  
 
- 编译工具

  - `cd /data/raspberry_pi4/toolchain_staging/patches/binutils/2.31.1`

  - `scp pi@targetip:/usr/src/binutils/patches/129_multiarch_libpath.patch .`
  
  - `cd /data/raspberry_pi4`
  
  - `ct-ng list-samples`
  
   > Status  Sample name  
     [G...]   aarch64-rpi3-linux-gnu  
     [G..X]   aarch64-unknown-linux-android  
     [G...]   aarch64-unknown-linux-gnu  
     [G...]   aarch64-unknown-linux-uclibc  
     [G...]   alphaev56-unknown-linux-gnu  

  - 生成配置: `ct-ng aarch64-rpi3-linux-gnu`
  
       > cat .config
   
  - 配置选项: `ct-ng menuconfig`
       
       > [pi4.config](pi4.config)  
       > mv pi4.config .config  (注意pi4kernel为5.4.51, menu没有,手动修改CT_LINUX_VERSION="5.4.51"
  
  - 编译:
  
     - `export DEB_TARGET_MULTIARCH=aarch64-rpi3-linux-gnu`
     - `unset LD_LIBRARY_PATH
     - `ct-ng build`
     
  - sysroot (根据情况选择, 最好不要做):
  
      - `cd aarch64-rpi3-linux-gnu/aarch64-rpi3-linux-gnu/sysroot`
      - `rsync -rzLR --safe-links pi@target_ip:/lib/ ./`
      - `rsync -rzLR --safe-links pi@target_ip:/usr/lib/ ./`
      - `rsync -rzLR --safe-links pi@target_ip:/usr/include ./`

In [ ]:
apt-get install qemu

## 编译Torch

### 获取源

- `git cloen --depth 1 https://github.91chifun.workers.dev//https://github.com/pytorch/pytorch.git`
- `cd pytorch`
- `sed -i 's/github.com\//github.com.cnpmjs.org\//g' .gitmodules`
- `git submodule sync`
- `git submodule update --init --recursive`  // 可以中间暂停, 修改github.com
- `cd .git/modules/third_part`
- 查看子模块, 修改(手动): ```grep -l "github.com\/" `find . -name "config"` ```


### 编译

  - 依赖: [pytorch source](https://github.com/pytorch/pytorch#from-source)
        
   ```
   pip3 install numpy ninja pyyaml mkl mkl-include setuptools cmake cffi \
           typing_extensions future six requests dataclasses --user｀
   ```
   
https://nmilosev.svbtle.com/compling-arm-stuff-without-an-arm-board-build-pytorch-for-the-raspberry-pi

https://github.com/nmilosev/pytorch-arm-builds/

## 编译vision

```
git clone --depth 1  https://github.com.cnpmjs.org/pytorch/vision.git
python3 setup.py install --user
```

## 编译wiringpi

```
git clone --depth 1 https://hub.fastgit.org/WiringPi/WiringPi.git
./build
```


## 编译gpiozero

```
git clone --depth 1 https://hub.fastgit.org/gpiozero/gpiozero.git
python3 setup install --user
```

`sudo apt install python3-rpi.gpio`

## GPIO

> pi@raspberrypi:~ $ gpio readall
```
 +-----+-----+---------+------+---+---Pi 4B--+---+------+---------+-----+-----+
 | BCM | wPi |   Name  | Mode | V | Physical | V | Mode | Name    | wPi | BCM |
 +-----+-----+---------+------+---+----++----+---+------+---------+-----+-----+
 |     |     |    3.3v |      |   |  1 || 2  |   |      | 5v      |     |     |
 |   2 |   8 |   SDA.1 |   IN | 1 |  3 || 4  |   |      | 5v      |     |     |
 |   3 |   9 |   SCL.1 |   IN | 1 |  5 || 6  |   |      | 0v      |     |     |
 |   4 |   7 | GPIO. 7 |   IN | 1 |  7 || 8  | 1 | IN   | TxD     | 15  | 14  |
 |     |     |      0v |      |   |  9 || 10 | 1 | IN   | RxD     | 16  | 15  |
 |  17 |   0 | GPIO. 0 |  OUT | 1 | 11 || 12 | 0 | OUT  | GPIO. 1 | 1   | 18  |
 |  27 |   2 | GPIO. 2 |   IN | 0 | 13 || 14 |   |      | 0v      |     |     |
 |  22 |   3 | GPIO. 3 |   IN | 0 | 15 || 16 | 0 | IN   | GPIO. 4 | 4   | 23  |
 |     |     |    3.3v |      |   | 17 || 18 | 0 | IN   | GPIO. 5 | 5   | 24  |
 |  10 |  12 |    MOSI |   IN | 0 | 19 || 20 |   |      | 0v      |     |     |
 |   9 |  13 |    MISO |   IN | 0 | 21 || 22 | 0 | IN   | GPIO. 6 | 6   | 25  |
 |  11 |  14 |    SCLK |   IN | 0 | 23 || 24 | 1 | IN   | CE0     | 10  | 8   |
 |     |     |      0v |      |   | 25 || 26 | 1 | IN   | CE1     | 11  | 7   |
 |   0 |  30 |   SDA.0 |   IN | 1 | 27 || 28 | 1 | IN   | SCL.0   | 31  | 1   |
 |   5 |  21 | GPIO.21 |   IN | 1 | 29 || 30 |   |      | 0v      |     |     |
 |   6 |  22 | GPIO.22 |   IN | 1 | 31 || 32 | 0 | IN   | GPIO.26 | 26  | 12  |
 |  13 |  23 | GPIO.23 |   IN | 0 | 33 || 34 |   |      | 0v      |     |     |
 |  19 |  24 | GPIO.24 |   IN | 0 | 35 || 36 | 0 | IN   | GPIO.27 | 27  | 16  |
 |  26 |  25 | GPIO.25 |   IN | 0 | 37 || 38 | 0 | IN   | GPIO.28 | 28  | 20  |
 |     |     |      0v |      |   | 39 || 40 | 0 | IN   | GPIO.29 | 29  | 21  |
 +-----+-----+---------+------+---+----++----+---+------+---------+-----+-----+
 | BCM | wPi |   Name  | Mode | V | Physical | V | Mode | Name    | wPi | BCM |
 +-----+-----+---------+------+---+---Pi 4B--+---+------+---------+-----+-----+
```

### LCD

```
sudo apt-get install -y python-smbus
sudo apt-get install -y i2c-tools
```


> pi@raspberrypi:~/demo $ sudo i2cdetect -y 1
```
     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:          -- -- -- -- -- -- -- -- -- -- -- -- -- 
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
20: -- -- -- -- -- -- -- 27 -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
40: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
70: -- -- -- -- -- -- -- --                         
```


## Camera (Q)

[aarch64 no mmal](https://github.com/raspberrypi/userland/issues/630)

https://www.geeks3d.com/hacklab/20200308/python-3-and-opencv-part-3-how-to-read-the-webcam-with-opencv-cv2/

http://datahacker.rs/001-how-to-read-a-video-and-access-a-webcam-with-opencv-in-python/

- 使能: `sudo raspi-config`

- ~~工具: `git clone --depth 1 https://github.com.cnpmjs.org/raspberrypi/userland.git`~~

## QA:

### 解决树莓派公钥

[更多](https://www.jianshu.com/p/8a1921413fad)

```
gpg --keyserver  keyserver.ubuntu.com --recv-keys 9165938D90FDDD2E
gpg --export --armor  9165938D90FDDD2E | sudo apt-key add -
```

### 安装Pillow错误

[依赖](https://pillow.readthedocs.io/en/latest/installation.html)

```
sudo apt-get install libtiff5-dev libjpeg62-turbo-dev libopenjp2-7-dev zlib1g-dev \
    libfreetype6-dev liblcms2-dev libwebp-dev tcl8.6-dev tk8.6-dev python3-tk \
    libharfbuzz-dev libfribidi-dev libxcb1-dev
```

### PyTorch packages for ARM64

[更多](http://mathinf.com/pytorch/arm64/)

`pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html`

### build cmake (pip 版本问题)

```
 Failed to build cmake
  matplotlib 3.3.3 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
  Installing collected packages: setuptools, wheel, pyparsing, six, packaging, distro, scikit-build, cmake, pip, numpy
    Running setup.py install for cmake: started
      Running setup.py install for cmake: finished with status 'error'
      Complete output from command /usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-8818b86w/cmake/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-v2b8zye8/install-record.txt --single-version-externally-managed --prefix /tmp/pip-build-env-4lmwqz8j --compile:
      Traceback (most recent call last):
        File "/tmp/pip-build-env-nbaxyom6/lib/python3.7/site-packages/skbuild/setuptools_wrap.py", line 560, in setup
          cmkr = cmaker.CMaker(cmake_executable)
        File "/tmp/pip-build-env-nbaxyom6/lib/python3.7/site-packages/skbuild/cmaker.py", line 95, in __init__
          self.cmake_version = get_cmake_version(self.cmake_executable)
        File "/tmp/pip-build-env-nbaxyom6/lib/python3.7/site-packages/skbuild/cmaker.py", line 82, in get_cmake_version
          "Problem with the CMake installation, aborting build. CMake executable is %s" % cmake_executable)
  
      Problem with the CMake installation, aborting build. CMake executable is cmake
```

解决:

```
python3 -m pip install --upgrade pip
pip3 install cmake --user
```

### OpenBLAS with USE_OPENMP=1

```
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option
```

[Report error when ATEN_THEADING is OMP and USE_OPENMP is turned off.](https://github.com/pytorch/pytorch/pull/40146)

### Opencv-Python

https://www.pyimagesearch.com/2019/09/16/install-opencv-4-on-raspberry-pi-4-and-raspbian-buster/

https://www.learnopencv.com/build-and-install-opencv-4-for-raspberry-pi/

- 增加swap大小 `vi /etc/dphys-swapfile` CONF_SWAPSIZE=2048
- sudo /etc/init.d/dphys-swapfile stop`
- sudo /etc/init.d/dphys-swapfile start`

```
# Build tools:
sudo apt-get install -y build-essential cmake

# GUI (if you want to use GTK instead of Qt, replace 'qt5-default' with 'libgtkglext1-dev' and remove '-DWITH_QT=ON' option in CMake):
sudo apt-get install -y qt5-default libvtk6-dev

# Media I/O:
sudo apt-get install -y zlib1g-dev libjpeg-dev libwebp-dev libpng-dev libtiff5-dev libjasper-dev libopenexr-dev libgdal-dev

# Video I/O:
sudo apt-get install -y libdc1394-22-dev libavcodec-dev libavformat-dev libswscale-dev libtheora-dev libvorbis-dev libxvidcore-dev libx264-dev yasm libopencore-amrnb-dev libopencore-amrwb-dev libv4l-dev libxine2-dev

# Parallelism and linear algebra libraries:
sudo apt-get install -y libtbb-dev libeigen3-dev

# Python:
sudo apt-get install -y python-dev python-tk python-numpy python3-dev python3-tk python3-numpy

cmake -D CMAKE_BUILD_TYPE=RELEASE \
    -D CMAKE_INSTALL_PREFIX=/usr/local \
    -D OPENCV_EXTRA_MODULES_PATH=~/opencv_contrib/modules \
    -D ENABLE_NEON=ON \
    -D ENABLE_VFPV3=ON \
    -D BUILD_TESTS=OFF \
    -D INSTALL_PYTHON_EXAMPLES=OFF \
    -D OPENCV_ENABLE_NONFREE=ON \
    -D CMAKE_SHARED_LINKER_FLAGS=-latomic \
    -D BUILD_EXAMPLES=OFF ..
```

[caffe2 onnx](https://detectron2.readthedocs.io/tutorials/deployment.html)

https://medium.com/@stonepreston/how-to-cross-compile-a-cmake-c-application-for-the-raspberry-pi-4-on-ubuntu-20-04-bac6735d36df

###  cv2.error

```
 cv2.imshow("frame",frame) 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
cv2.error: OpenCV(4.4.0) /tmp/pip-install-yw9tpjkf/opencv-python/opencv/modules/highgui/src/window.cpp:651: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
```

**使用matplotlib**

```
Traceback (most recent call last):
  File "cap_video.py", line 20, in <module>
    if cv2.waitKey(100) & 0xFF == ord('q'):
cv2.error: OpenCV(4.4.0) /tmp/pip-install-yw9tpjkf/opencv-python/opencv/modules/highgui/src/window.cpp:717: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvWaitKey'
```

### no-GUI backend

```
UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
```

remove: matplotlib.use('Agg')